In [1]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import time
import diffusionstuff7 as ds
from copy import copy as dup
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
from numba import int64

#for 3d plots
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
#for 3d plots
import matplotlib.animation as animation

#for saving simulations
import pickle

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [6]:
# Graphics parameters
%matplotlib widget
ticklabelsize = 15
fontsize = 15
linewidth = 2
fignum = 0

In [4]:
class Simulation():
    """Simulation class for integratable differential equation models
    
    Attributes:
    ----------
    model (func): integratable differential equation model
    shape (tuple): shape of initial condition
    method (str): integration method
    atol (float): absolute tolerance
    rtol (float): relative tolerance
    
    Internal attributes:
        _plot: matplotlib figure
        _animation: matplotlib animation
        _results: results of simulation
    
    Methods:
    ----------
    run(): runs simulation
    plot(): plots results of simulation
    animate(): animates results of simulation
    results(): returns results of simulation (handles running if necessary)
    save(): saves simulation object to file
    load(): loads simulation object from file
    
    @author: Max Bloom 
        contact: mbloom@pugetsound.edu
    """

    def __init__(self, model, shape, method= "LSODA", atol= 1e-6, rtol= 1e-6):
        """Initialize the Simulation
        Parameters
        ----------
        model (func): integratable differential equation model
        shape (tuple): shape of initial condition

        method (str): integration method
        atol (float): absolute tolerance
        rtol (float): relative tolerance
        """
        #solve_ivp arguments
        self.model = model #integratable differential equation model 
        #self.t_span = t_span #time span
        #self.y0 = y0 #initial conditions 
        #self.t_eval = t_eval
        self.method = method #default to emulate odeint
        self.atol = atol #default absolute tolerance
        self.rtol = rtol #default relative tolerance

        #internal attributes
        self._plot = None #matplotlib figure
        self._animation = None #matplotlib animation
        self._results = {None} #solve_ivp dictionary of results

        pass

    def run(self) -> None:
        try:#try to run simulation and package results
            self._results = solve_ivp(self.model, self.t_span, self.y0, t_eval=self.t_eval, method=self.method, atol=self.atol, rtol=self.rtol, args=self._args)
            #TODO: save parameters that the model needs
            # Nice=Nicekeep, Fliq=Fliqkeep,
            #                     x=x, t=tkeep, 
            #                     Nbar=Nbar, Nstar=Nstar,
            #                     sigma0=sigma0, c_r=c_r, D=D, L=L, 
            #                     nu_kin=nu_kin, nu_kin_ml=nu_kin_ml, 
            #                     sigmastepmax=sigmastepmax, sigmastepstyle=sigmastepstyle,
            #                     ykeep_0Darr=ykeep_0Darr,
            #                     tkeep_0D=tkeep_0D,  
            #                     dtmaxtimefactor = dtmaxtimefactor,
            #                     deltaT = deltaT
        except Exception as e:
            print(e)
            print('Error in simulation')
        pass
    

    def plot(self):# -> matplotlib_figure: #TODO:
        """ plot results of simulation, returns matplotlib figure """
        if self._plot == None:
            #create plot of results
            my_results = self.results()
            _plot = plt.figure()

            ax = plt.axes()
            plt.ion()
            plt.show()
        return self._plot
    
    def animate(self, proportionalSpeed=True):# -> matplotlib_figure: #TODO:
        if self._animation == None:
            #create animation of results
            def nextFrame():
                pass
            my_results = self.results()
            #if proportionalSpeed:#TODO: scale interval to make length of gif/mp4 be 10 seconds, scaling speed of animation by factor proportional to length of simulation
                #interval = 

            _animation = animation.FuncAnimation(self._plot, nextFrame, interval=10)
            #create animation of results
        return self._animation

    #completed functions below here
    def results(self) -> dict:
        """ returns results of simulation (handles running if necessary) """
        if self._results == {None}:
            self.run()
        return self._results

    def save(self, _id = []) -> None:
        """ saves pickle of simulation object 
        
        args:
            _id: list of strings to append to filename: what makes this simulation unique
        """
        # Saving these results to file
        #if Nice[0] > 100000: #what is this nesh?
        #    Nice -= 100000
        _id = str('_'+i for i in _id)
        filename = self.model.__name__+'_simulation'+_id+'.pkl'
        with open(filename, 'wb') as f:
            print("saving to", f)
            pickle.dump(self, f)
        pass

    def load(self, filename) -> None:
        """ loads pickle of simulation object """
        with open(filename, 'rb') as f:
            self = pickle.load(f)
        pass
    
    def save_animation(self, filename, filetype) -> None:
        """ saves animation of simulation object """
        try:
            if filetype == 'mp4':
                Writer = animation.writers['ffmpeg']
                writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
            elif filetype == 'gif':
                Writer = animation.writers['imagemagick']
                writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
            else:
                print('filetype not supported')
                return
        except Exception as e:
            print(e)
            print('Error creating animation file writer')
            return
        try:
            self.animate().save(filename+'.'+filetype, writer=writer)
        except Exception as e:
            print(e)
            print('Error saving animation')
            return
        pass

In [5]:
def runSimulations(params_array) -> dict:
    """
    Run the simulations and return the solve_ivp results dictionary.
    """
    results_array = []
    for params in params_array:
        
        sim = Simulation(model, shape, method=method, atol=atol, rtol=rtol)
        sim.run()
        results_array[params] = sim.results()
    return results_array